In [18]:
import numpy as np
from scipy import constants as const
import pickle as pkl

# Calculation of scaling constant using PK-4-relevant parameters

In order to tie the simulation to real a dusty plasma system we normalize space and time according to the electron Debye length (`lambda_De` or $\lambda_{De}$, the relevant Yukawa screening length) and the plasma *dust* frequency in Hertz (`f_pd` or $f_{pd}$). These quantities are calculated below. The "scaling constant" comes from collecting all these values from the normalizations along with any constants in the equation of motion, so
$$
\begin{equation*}
    m_d\frac{d^2r}{dt^2}=\frac{q_d^2}{4\pi\varepsilon_0}
    \left[
        \frac{e^{-r/\lambda_{Di}}}{\lambda_{Di}r} + \frac{e^{-r/\lambda_{Di}}}{r^2}
    \right],
\end{equation*}
$$
where $m_d$ is the mass of the dust, $r$ is the interparticle spacing, and $q_d$ is the dust charge, becomes
$$
\begin{equation*}
    \frac{d^2\hat{r}}{(dt^{*})^2}=A
    \left[
        \frac{e^{-\hat{r}}}{\hat{r}} + \frac{e^{-\hat{r}}}{\hat{r}^2}
    \right],
\end{equation*}
$$
where $\hat{r}=r/\lambda_{Di}$, $t^*=\omega_{pd}t$, and $A=\frac{q_d^2}{4\pi\varepsilon_0 m_d \omega_{pd}^2 \lambda_{Di}^3}$ is what we refer to as the "scaling constant."

In [22]:
# scaling constant
m_d = 3.03e-14  # dust mass in kg
mu = m_d / 2    # reduced mass
n_d = 1e11      # dust density in m^-3
n_e = 2.81e14   # electron density in m^-3
q_d = 1e4 * const.e   # dust charge


kT_e_eV = 7.75  # T_e in eV from RSI paper
kT_e = kT_e_eV * const.e # 1.24e-18 # in Joules (converted from eV)

# calculate ion quantities
T_i = 293.15 # Room temp in K
kT_i = const.k * T_i
kT_i_eV = kT_i / const.e

# ion Debye length assuming quasineutrality
lambda_Di = ( ( const.epsilon_0 * kT_i ) / ( n_e * const.e**2 ) )**(1/2)

lambda_De = ( ( const.epsilon_0 * kT_e ) / ( n_e * const.e**2 ) )**(1/2)
omega_pd = np.sqrt( ( n_d * q_d**2 ) / ( const.epsilon_0 * m_d ) )
f_pd = omega_pd / (2 * np.pi)

A_De = q_d**2 / (4 * np.pi * const.epsilon_0 * mu * lambda_De**3 * f_pd**2)
print("scaling constant with electron Debye length and plasma freq in Hz:", A_De)

A = q_d**2 / (4 * np.pi * const.epsilon_0 * mu * lambda_Di**3 * omega_pd**2)
print("scaling constant with ion Debye length and plasma freq in rad/s:", A)

A_fpd = q_d**2 / (4 * np.pi * const.epsilon_0 * mu * lambda_Di**3 * f_pd**2)
print("scaling constant with ion Debye length and plasma freq in Hz:", A)

scaling constant with electron Debye length and plasma freq in Hz: 0.033390989559415385
scaling constant with ion Debye length and plasma freq in rad/s: 4.544937607428946
scaling constant with ion Debye length and plasma freq in Hz: 4.544937607428946


Save the scaling constant in the working directory using pickle.

In [23]:
with open('../scaling_const.float', 'wb') as f:
    pkl.dump(A, f, pkl.HIGHEST_PROTOCOL)

In [15]:
print("electron Debye length", lambda_De, "m")
print("ion Debye length:", lambda_Di, "m")
lambda_D = lambda_De*lambda_Di / ( lambda_De + lambda_Di )
print("plasma Debye length:", lambda_D, "m")

# in units of PK-4 camera pixels
px_dim = 14e-6
lambda_De_px = lambda_De / px_dim
print("electron Debye length (px):", lambda_De_px)
lambda_Di_px = lambda_Di / px_dim
print("ion Debye length (px):", lambda_Di_px)

electron Debye length 0.0012354127099723797 m
ion Debye length: 7.058100095522309e-05 m
plasma Debye length: 6.676652799554636e-05 m
electron Debye length (px): 88.24376499802712
ion Debye length (px): 5.041500068230221


We can interpret this scaling constant as proportional to a ratio of energies, particularly the electrostatic Coulomb potential energy and the electron thermal kinetic energy. We show this here:
$$
\begin{align*}
    A &= \frac{q_d^2}{4\pi\varepsilon_0\lambda_{Di}} \frac{1}{m_d \omega_{pd}^2\lambda_{Di}^2} \\
      &= \frac{q_d^2}{4\pi\varepsilon_0\lambda_{Di}} \frac{1}{m_d} \frac{\varepsilon_0 m_d}{n_d q_d^2} \frac{n_i e^2}{\varepsilon_0 k_B T_i} \\
      &= \frac{n_i}{n_d Z_d^2} \frac{q_d^2}{4\pi\varepsilon_0\lambda_{Di}} \frac{1}{k_B T_i}
\end{align*}
$$
where $k_B$ is the Boltzmann constant, $e$ is the fundamental charge, and $Z_d=\frac{q_d}{e}$.
$$
\begin{equation*}
  A \propto \frac{\epsilon_{C,d}}{\epsilon_{th,i}}
\end{equation*}
$$
with $\epsilon_{C,d}=\frac{q_d^2}{4\pi\varepsilon_0\lambda_{Di}}$ and $\epsilon_{th,i}=k_B T_i$